In [1]:
from dataloader import Dataset
from utils import * 

data = Dataset("Stage_1_Data_ver._3/stage1/box")
data.load_data()

visibility = data.visibility

In [2]:
images_to_add, appended_ids = data.computeSceneGraph()

In [123]:

indices    = np.logical_and(visibility[:,appended_ids[0]], visibility[:,appended_ids[1]])
pts1, pts2 = data.points[indices, appended_ids[0]], data.points[indices, appended_ids[1]]
E, best_criterion = estimate_essential_matrix(pts1, pts2, data.K)
rotations, translations = extract_cam_pose(E, data.K)
rot, trans = decomposeEssentialMat(E)
points_3d = []
for i in range(4):
    P1 = computeProjectionMatrix(data.K, np.identity(3), np.zeros((3,)))
    P2 = computeProjectionMatrix(data.K, rotations[i], translations[i])
    points_3d.append(linearTriangulation(pts1, pts2, P1, P2))

X,R,C = cheiralityCondition(points_3d, rotations, translations)

X_reconstructed   = np.zeros((data.points.shape[0], 3))
reconstructed_ind = np.zeros((data.points.shape[0], 1))

X_reconstructed[indices] = X
reconstructed_ind[(X_reconstructed[:,2] > 0)] = 1

camera_rotations    = []
camera_translations = []
camera_rotations.append(np.eye(3))
camera_rotations.append(R)
camera_translations.append(np.zeros((3,)))
camera_translations.append(C)


In [124]:
X.min()

-5061.461029505171

In [125]:
for next_image in range(2, 3):

    visibility_points = data.visibility[:, appended_ids[next_image]][...,None]
    new_points        = data.points[:,appended_ids[next_image]]
    indices           = np.logical_and(reconstructed_ind, visibility_points)[:,0]

    X_new       = X_reconstructed[indices]
    corr_points = new_points[indices]

    R_new, C_new = DLT_PnP(corr_points, X_new, data.K)

    camera_rotations.append(R_new)
    camera_translations.append(C_new)
    overlap = []
    for previous_images in range(next_image):
        
        indices = np.logical_and(np.logical_and(visibility[:,appended_ids[previous_images]][...,None], 
                                                visibility[:,appended_ids[next_image]][...,None]), 1 - reconstructed_ind)[:,0]
        overlap.append(np.logical_and(visibility[:,appended_ids[previous_images]][...,None],visibility[:,appended_ids[next_image]][...,None]))
        pts1, pts2 = data.points[indices, appended_ids[previous_images]], data.points[indices, appended_ids[next_image]]
        P1 = computeProjectionMatrix(data.K, camera_rotations[previous_images], camera_translations[previous_images])
        P2 = computeProjectionMatrix(data.K, R_new, C_new)
        points_3d = linearTriangulation(pts1, pts2, P1, P2)
        X_reconstructed[indices]   = points_3d
        reconstructed_ind[indices] = 1
    
    break
    reconstructed_ind[(X_reconstructed[:,2] <0)] = 0
    
    

    

(12613, 1) (12613, 1) (12613,) [False False False ... False False False]


In [132]:
def getObservationsIndexAndVizMat(X_found, filtered_feature_flag, nCam, appended_ids):
    # find the 3d points such that they are visible in either of the cameras < nCam
    bin_temp = np.zeros((filtered_feature_flag.shape[0]), dtype = int)
    for n in range(nCam + 1):
        bin_temp = np.logical_or(bin_temp,  filtered_feature_flag[:,appended_ids[n]])
    

    X_index = np.where(np.logical_and(X_found.reshape(-1) , (bin_temp)))
    visiblity_matrix = X_found[X_index].reshape(-1,1)
    for n in range(nCam + 1):
        visiblity_matrix = np.hstack((visiblity_matrix, filtered_feature_flag[X_index, appended_ids[n]].reshape(-1,1)))

    o, c = visiblity_matrix.shape
    return X_index, visiblity_matrix[:, 1:c]

In [133]:
ind, viz = getObservationsIndexAndVizMat(reconstructed_ind, visibility, 2, appended_ids)

In [157]:
def computeCurrentVisibility(visibility, reconstructed_ind, number_of_views, appended_ids):

    X_indices = (reconstructed_ind == 1)[:,0]
    current_visibility = visibility[:,appended_ids[:number_of_views]][X_indices]
    
    return X_indices, current_visibility

# def computeCurrent2DPoints(X_indices, points_2d, appended_ids):

    
    

In [156]:
def get2DPoints(X_index, visiblity_matrix, feature_x, appended_ids):
    pts2D = []
    visible_feature_x = feature_x[X_index]
    h, w = visiblity_matrix.shape
    for i in range(h):
        for j in range(w):
            if visiblity_matrix[i,j] == 1:
                pt = np.hstack((visible_feature_x[i,j,0], visible_feature_x[i,j,1]))
                pts2D.append(pt)
    return np.array(pts2D).reshape(-1, 2) 

In [158]:
X_indices, current_visibility = computeCurrentVisibility(visibility, reconstructed_ind, 3, appended_ids)

In [159]:
current_2d_points = get2DPoints(X_indices, current_visibility, data.points, appended_ids)

In [137]:
X_indices = (reconstructed_ind == 1)[:,0]
current_visibility = visibility[X_indices, appended_ids[:3]] 

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (1638,) (3,) 

array([1426., 1351., 1386.])

In [88]:
np.logical_or(overlap[1][:,0],initial).sum()


1390

In [122]:
viz

324.0

In [95]:
current_visibility.shape

(12613, 3)

In [40]:
visibility[:,appended_ids[:3]].sum(axis= 0 )

array([1953., 2055., 1892.])

In [47]:
current_visibility.shape

(12613, 3)

In [98]:
ind[0].shape

(1677,)

In [119]:
visibility[reconstructed_ind[:,0] == 1, appended_ids[0]].sum()

1465.0

In [100]:
A = lil_matrix(3,4)

In [101]:
A

<1x1 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in List of Lists format>